<a href="https://colab.research.google.com/github/maryamnassirian/AI_in_BiomedicalData/blob/main/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import scipy. io
import numpy as np
import torch
fr_samp= 200
path= r'./data/'
file_list= os.listdir(path)
data= torch.tensor([])
mark= torch.tensor([])
data_list= []
marker_list= []
for file_name in file_list:
  if file_name.endswith(' .mat'):
    file_path= os.path.join(path, file_name)
    mat_data= scipy.io.loadmat(file_path)
    data_array= np.array(mat_data['o']['data'][0, 0])
    marker_array= np.array(mat_data['o']['marker'][0, 0])
    torch_data= torch.from_numpy(data_array)
    torch_marker= torch.from_numpy(marker_array)
    data= torch.cat((data, torch_data), dim=0)
    mark= torch.cat((mark, torch_marker), dim=0)
mark= mark[:,0]
print(mark.shape)
print(data.shape)


In [ ]:
pr= 0
cl_l_ind= []
cl_2_ind= []
cl_3_ind= []
cl_4_ind= []
cl_5_ind= []

for i in range(mark.shape[0]):
  nx= mark[i]
  if pr != nx:
    if nx== 1:
      cl_l_ind.append(i)
    elif nx== 2:
      cl_2_ind.append(i)
    elif nx== 4:
      cl_3_ind.append(i)
    elif nx== 5:
      cl_4_ind.append(i)
    elif nx== 6:
      cl_5_ind.append(i)
  pr= nx
print(len(cl_l_ind), len(cl_2_ind), len(cl_3_ind), len(cl_4_ind), len(cl_�

separation of trials

In [ ]:
def trial_sepration(data, mark_indx, length= fr_samp):
  output= torch.tensor([])
  for index in mark_indx:
    output= torch.cat((output, data[index: index+length, :].unsqueeze(
  return output

In [ ]:
def data_mark_maker(cl_ind, new_mark, dataset= data):
    cls_data= trial_sepration(dataset, cl_ind)
    cls_mark= torch.full((cls_data.shape), new_mark, dtype=torch.long)[:,�
    return cls_data, cls_mark

In [ ]:
els_l_data, cls_l_mark= data_mark_maker(cl_l_ind, 0)
cls_2_data, els_2_mark= data_mark_maker(cl_2_ind, 1)
els_3_data, els_3_mark= data_mark_maker(cl_3_ind, 2)
cls_4_data, cls_4_mark= data_mark_maker(cl_4_ind, 3)
els_5_data, els_5_mark= data_mark_maker(cl_5_ind, 4)

In [ ]:
del cl_l_ind, cl_2_ind, cl_3_ind, cl_4_ind, cl_5_ind

concat cls

In [ ]:
data= torch.cat((cls_l_data, cls_2_data, cls_3_data, cls_4_data, els_5_data
mark= torch.cat((cls_l_mark, cls_2_mark, cls_3_mark, cls_4_mark, els_5_mark

In [ ]:
del cls_l_data, cls_2_data, cls_4_data, cls_5_data, cls_3_data
del cls_l_mark, cls_2_mark, cls_4_mark, cls_5_mark, cls_3_mark

In [ ]:
num_sample= mark.shape[0]
rnd_perm= torch.randperm(num_sample)

data= data[rnd_perm].float()
mark= mark[rnd_perm].long()

save data

In [ ]:
data= data.unsqueeze(1).permute(0,1,3,2)
torch.save(data, 'data. pth' )
torch.save(mark, 'mark. pth')

print(data.shape)
print(mark.shape)

split

In [ ]:
from sklearn.model_selection import train_test_split
train ratio= 0.8

x_train, x_test, y_train, y_test= train_test_split(data, mark, train_size=
x_train, x_valid, y_train, y_valid= train_test_split(x_train, y_train, trc
print( 'train: ',  x_train.shape, y_train.shape)
print( 'valid: ',  x_valid.shape, y_valid.shape)
print('test: ',  x_test.shape, y_test.shape)

data loader

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
train_batch_size= 330
valid_batch_size= 330

train_dataset= TensorDataset(x_train, y_train)
valid_dataset= TensorDataset(x_valid, y_valid)
test_dataset= TensorDataset(x_test, y_test)

train_loader= DataLoader(train_dataset, batch_size= train_batch_size, shu)
valid_loader= DataLoader(valid_dataset, batch_size= valid_batch_size, shu)
test_loader= DataLoader(test_dataset, batch_size= valid_batch_size, shuf)

print("train batch size:",train_loader.batch_size, ",num of batch:", len)
print("valid batch size:",valid_loader.batch_size, ",num of batch:", len)
print("test batch size:",test_loader.batch_size, ",num of batch:", len)



In [ ]:
x,y= next(iter(train_loader))
print(x.shape, y.shape)

model

hyperparameters

In [ ]:
num class= 5
num_input= 1
channel= 22
signal_length= 200
fs= 200

Fl= 8
D= 3
F2= D*F1
kernel_size_1= (1,round(fs/2))
kernel_size_2= (channel, 1)
kernel_size_3= (1, round(fs/8))
kernel_size_4= (1, 1)

kernel_avgpool_1= (1,4)
kernel_avgpool_2= (1,8)
dropout_rate= 0.2
ks0= int(round((kernel_size_1[0]-1)/2))
ks1= int(round((kernel_size_1[1]-1)/2))
kernel_padding_1= (ks0, ks1-1)
ks0= int(round((kernel_size_3[0]-1)/2))
ks1= int(round((kernel_size_3[1]-1)/2))
kernel_padding_3= (ks0, ks1)

defining the base class

In [ ]:
import torch.nn as nn
class EEGNet(nn.Module):
def __init__(self):
super().__init__()
# Layer 1
self.conv2d= nn.Conv2d(num_input, F1, kernel_size_1, padding=kerne
self.Batch_normalization_1= nn.BatchNorm2d(Fl)
# Layer 2
self.Depthwise_conv2D= nn.Conv2d(Fl, D*F1, kernel_size_2, groups=
self.Batch_normalization_2= nn.BatchNorm2d(D*Fl)
self.Elu= nn.ELU()
self.Average_pooling2D_1= nn.AvgPool2d(kernel_avgpool_1)
self.Dropout= nn.Dropout2d(dropout_rate)
# Layer 3
self.Separable_conv2D_depth= nn.Conv2d(D*F1, D*F1, kernel_size_3,
padding= kernel_padding_3,
self.Separable_conv2D_point= nn.Conv2d(D*F1, F2, kernel_size_4)
self.Batch_normalization_3= nn.BatchNorm2d(F2)
self.Average_pooling2D_2= nn.AvgPool2d(kernel_avgpool_2)
# Layer 4
self.Flatten= nn.Flatten()
self.Dense= nn.Linear(F2* round(signal_length/32), num_class)
self.Softmax= nn.Softmax(dim= 1)


def forward(self, x):
    # Layer 1
    y= self.Batch_normalization_1(self.conv2d(x))
    # Layer 2
    y= self.Batch_normalization_2(self.Depthwise_conv2D(y))
    v= self. Elu (y)
    y= self.Dropout(self.Average_pooling2D_1(y))
    # Layer 3
    y= self.Separable_conv2D_depth(y)
    y= self.Batch_normalization_3(self.Separable_conv2D_point(y))
    v= self. Elu (y)
    y= self.Dropout(self.Average_pooling2D_2(y))
    # Layer 4
    y= self.Flatten(y)
    y= self.Dense(y)
    y= self.Softmax(y)

    return y

In [ ]:
import torch.optim as optim

model= EEGNet()
eta= 0.01
loss_fn= nn.CrossEntropyloss()
optimizer= optim.NAdam(model.parameters(), lr= eta)

train

one epoch

In [ ]:
from torchmetrics import Accuracy

def train_one_epoch(model, train_loader, loss_fn, optimizer):
    model.train()
    loss_train= AverageMeter()
    acc_train= Accuracy(task= "multiclass", num_classes= num_class)

    for i, (inputs, targets) in enumerate(train_loader):
        outputs= model(inputs)
        loss= loss_fn(outputs, targets)

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        optimizer.zero_grad()
        loss_train.update(loss.item())
        acc_train(outputs, targets.int())
return model, loss_train.avg, acc_train.compute().item()

In [ ]:
from torchmetrics import F1Score

def test_one_epoch(model, test_loader, loss_fn, optimizer):
    model. eval ()
    loss_test= AverageMeter()
    acc_test= Accuracy(task= "multiclass", num_classes= num_class)
    f1_test= F1Score(task= "multiclass", num_classes= num_class)
    for i, (inputs, targets) in enumerate(test_loader):
        outputs= model(inputs)
        loss= loss_fn(outputs, targets)
        loss_test.update(loss.item())
        acc_test(outputs, targets.int())
        f1_test(outputs, targets.int())
return model, loss_test.avg, acc_test.compute().item(), f1_test.compu1

util

In [ ]:
class AverageMeter(object):
"""Computes and stores the average and current value"""

    def __init__(self):
        self.reset()
    def reset(self):
        self.val = 0
        self.avg= 0
        self.sum= 0
        self.count = 0

def update(self, val, n=1):
        self.val= val
        self.sum+= val* n
        self.count+= n
        self.avg= self.sum / self.count

training

In [ ]:
num_epochs= 50
loss_train_hist, acc_train_hist= [], []
loss_valid_hist, acc_valid_hist= [], []
loss_test_hist, acc_test_hist= [], []


for epoch in range(num_epochs):
    model, loss_train, acc_train= train_one_epoch(model, train_loader, lo�
    model, loss_valid, acc_valid, _= test_one_epoch(model, valid_loader, J
    model, loss_test, acc_test, flscore= test_one_epoch(model, test_loader

    loss_train_hist.append(loss_train)
    acc_train_hist.append(acc_train)
    loss_valid_hist.append(loss_valid)
    acc_valid_hist.append(acc_valid)
    loss_test_hist.append(loss_test)
    acc_test_hist.append(acc_test)
    if (epoch%5== 0):
print(f'epoch {epoch}:')
print(f' Loss= {loss_train:.4}, Accuracy= {int(acc_train*100)}%')
print(f' Loss= {loss_valid:.4}, Accuracy= {int(acc_valid*100)}%')
print(f' Loss= {loss_test:.4}, Accuracy= {int(acc_test*100)}%')
print(f' f1 score= {(f1score*100):.4}\n')

learning curve
loss

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(num_epochs), loss_train_hist, 'b-', label= 'Train')
plt.plot(range(num_epochs), loss_valid_hist, 'k-', label= 'Valid')
plt.plot(range(num_epochs), loss_test_hist, 'g-', label= 'Test')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

accuracy

In [ ]:
plt.plot(range(num_epochs), acc_train_hist, 'b-', label= 'Train')
plt.plot(range(num_epochs), acc_valid_hist, 'k-', label= 'Valid')
plt.plot(range(num_epochs), acc_test_hist, 'g-', label= 'Test')
plt.xlabel('Epoch')
plt.ylabel('Acc')
plt.grid(True)
plt.legend()